# Character Recurrent Neural Network
- Mimicing Shakespeare's writing style
- Long short-term memory(LSTM)

![alt text](./LSTM.png)

## 1. Settings
### 1) Import required libraries

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
import unidecode
import string
import random
import re
import time, math

## 2) Hyperparameter

In [3]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002

## 2. Data
### 1) Prepare characters

In [4]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


### 2) Get text data

In [5]:
file = unidecode.unidecode(open('./data/shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115394


## 3. Functions for text processing
### 1) Random Chunk

In [6]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())

I will marry you.
Now, Kate, I am a husband for your turn;
For, by this light, whereby I see thy beauty,
Thy beauty, that doth make me like thee well,
Thou must be married to no man but me;
For I am he


### 2) Character to tensor

In [7]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)#.cuda()

print(char_tensor('ABCdef'))

Variable containing:
 36
 37
 38
 13
 14
 15
[torch.LongTensor of size 6]



### 3) Chunk into input & label

In [8]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3. Model & Optimizer

![alt text](./lstm_ex.jpg)

### 1) Model

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden,cell):
        out = self.encoder(input.view(batch_size,-1))
        out,(hidden,cell) = self.rnn(out,(hidden,cell))
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden,cell

    def init_hidden(self):
          
        hidden = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        cell = Variable(torch.zeros(num_layers,batch_size,hidden_size))#.cuda()
        
        return hidden,cell
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)#.cuda()

In [10]:
inp = char_tensor("A")
print(inp)
hidden,cell = model.init_hidden()
print(hidden.size())

out,hidden,cell = model(inp,hidden,cell)
print(out.size())

Variable containing:
 36
[torch.LongTensor of size 1]

torch.Size([1, 1, 100])
torch.Size([1, 100])


### 2) Loss & Optimizer

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

### 3) Test function

In [12]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden,cell = model.init_hidden()
    x = inp

    print(start_str,end="")
    for i in range(200):
        output,hidden,cell = model(x,hidden,cell)

        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

## 4. Train

In [13]:
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden,cell = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j]
        y,hidden,cell = model(x,hidden,cell)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n\n")


 Variable containing:
 4.5685
[torch.FloatTensor of size 1]
 

bnnv!e-O!Y~pR=9	vvB22&/9oa=	M)//inn"6I!]54Sq&-gK(.%eQF	V\,Kv
X)7%Qfj$	<h4z>&p[D*RM;;/vED>F=Xja]9-FtoA|gw<]ga_c	b:VoV':p71|_v#bHV''G5I;OgW3h>I=)PzYB$d T00QGEd\



 Variable containing:
 2.2626
[torch.FloatTensor of size 1]
 

b;Sh yel  ras is,
: cor omy pern dratl whre kean
shes cathd fo' hal ot ou yod indt
ubaId bat sovoamn souO
SC:
Ier ith war ceat Sisd ha,

Whek
 near anth forl tatr, tou to in tod gry ou ry hat ance wcer



 Variable containing:
 2.0533
[torch.FloatTensor of size 1]
 

bens!
al lou the prat atk yohor to byiondd I berengighend the the to bincesan, bucesald,
Co not tt miks nou fufre thirwa, me t my I palll not that re kaensd the gof o cus brall.
An and thing domesth yo



 Variable containing:
 2.1994
[torch.FloatTensor of size 1]
 

b in to shidisrs; ond the frelonconed, what culest to mriold becons tEato my to pure she will tit sepy this pandis for no om with ble sistruers bit renast ie worrich ficr


 Variable containing:
 1.8060
[torch.FloatTensor of size 1]
 

be thee spell they he blisper still make he seepuly had lead me bring of
pal weld dip we king towels,
And agnain of the was 'the lishs toull of the reparding of we spired,
And not Here the king of yet




 Variable containing:
 1.8845
[torch.FloatTensor of size 1]
 

brice my the now to spusention, sicune
That left apon your farence,
To to be friecle'ss the instrience,
That the gation me haw riunce
Is addone her chould,
And your light, sume, the would a just the wo



 Variable containing:
 1.6200
[torch.FloatTensor of size 1]
 

ble redurs the fould my simand for this tour stbe cauch the due,
That your we couls recey dise my camoun to that to couth sir.

BRUTUS:

MARIO:
Murner more hope with whie me of in a pepturn't, Vear for



 Variable containing:
 1.6872
[torch.FloatTensor of size 1]
 

b as to me hangs houss,
And thou encer'd Clancunce betring a kingnor, I brock the be eeve than untumnotaring in I world pargence agn

For marry as you will calless the mide thou will the stay of thou her brisal pusia dare Caustry,
How that he wiarsured his lought thou hurds a caus



 Variable containing:
 1.7518
[torch.FloatTensor of size 1]
 

bat that mine conter with yount'd
Than the compect-orn thou?

BUSENVOLIO:
He round thou struen it to by to chider sweet molling the chart eres:
My more not, it a durity,
There my guilath.

CORIOLANUS:




 Variable containing:
 1.5898
[torch.FloatTensor of size 1]
 

bur, a daughter awondly of this Jove with be or you dest I our whine shest nothing.

HERMIONE:
Nor thee his
sout Burther that nothing will distreat: men me the revenctow be blagg bout affainle thought 



 Variable containing:
 1.1486
[torch.FloatTensor of size 1]
 

batience: me lost are low me brevinest of
More not death our pase the gracy.

MENENIO:
What not slouls and more
Mine compent-againted a fear the grast the guithem
As my goods make are let meand his bot



 Variable containing:
 1.7038
[torch.FloatTen

A vonious and the can and when wear mine and shall bear would in the paties the condent,
And thy father, trial the good him,
An a time will the sory and comporn the stranced me of no to my l



 Variable containing:
 1.7482
[torch.FloatTensor of size 1]
 

bind baked was Hence can unot heel.

ANGELO:
Uncweep some promband the with thee of hings and kept not bose chance bord,'
By the disstands untenstly to his may you, Kather in the
some, when for beinger



 Variable containing:
 1.3370
[torch.FloatTensor of size 1]
 

be counter and whore follish? Peritiest?
If you threedander you with these of more own you one this,
a pighal, anger the priceng of your houd is is wound
A blood, for the stranger.

KING HENRY VII:
O, 



 Variable containing:
 1.9535
[torch.FloatTensor of size 1]
 

beford be them for that heel before,
Let still all my lord,
A sclow my ruch then, gentless will come to the sly master yarch retiseous in hurself
He could comforfious that
with he so ma word, the last 



 V


 Variable containing:
 1.4193
[torch.FloatTensor of size 1]
 

be not, but need
That bring the preised thy fale, I way not her and him
curposed my lord it from here now, patenous, adny
Thou do the mause. I have so joy?

ESCALUS:
I kneaven of your castle to such sp



 Variable containing:
 1.8585
[torch.FloatTensor of size 1]
 

bed
O, then I have that rast I will may his like to born my blood,
In hand in the misures stand life, I will greaf all!
How he too say, my man and that, well
In princes it it or compled
Camposinaning f



 Variable containing:
 1.6970
[torch.FloatTensor of size 1]
 

bent
Yet too let wear that he do all of myself with gaves therefore:
What shall be love that call you warness for it.

WARWICK:
My love honour rest dead,

LEONTES:
I as Mistree south. What's they sweet



 Variable containing:
 1.5052
[torch.FloatTensor of size 1]
 

batlebless with my day.

ANGELO:
God, my lord, gentince, so heaven of hid
And ye bow the hollain she head show.

BALTHASAS:
Aloding 

I what date, and strails any and know him in me sess
And the wee



 Variable containing:
 2.2589
[torch.FloatTensor of size 1]
 

best more trike of your keep

CAPULET:
Who, sight the peers? That stept,
The choops sinesty far alone would tell hear spoin
Amen the bloodant me is the son, what besands well hot
do thou do brine.

CLA



 Variable containing:
 1.6640
[torch.FloatTensor of size 1]
 

but our countries.

LAUDIO:
And treats a blood forior for thee,
The dare a cannot remes thenclence to then,
And her look agot son, sure hands.

DUKE VINCENTIO:
I sweet that I say she but is no like
Ire



 Variable containing:
 1.3390
[torch.FloatTensor of size 1]
 

bats, and
the may are stins see you gried: our man
Dar; when this think are strong in his tion, give so the man
hat I so I have I have are your grow.

KING RICHARD II:
Ah, the inlays the joyal our hang



 Variable containing:
 1.5742
[torch.FloatTensor of size 1]
 

bed strange.

CAMILLO:
This not not command to you.

PERDYCY:
I l

Intent fasing the frown the libercerning deder.

TRANIO:
One joy with the boud! a Lad



 Variable containing:
 1.8626
[torch.FloatTensor of size 1]
 

be is damish.

GLOUCESTER:
And nusio, foot be accome,
Where I swall not lischer, my mistress at here fastal you ears:
But Rome, I have thought you hays.
What we would here's sond.

PAULIHA:
Here come. 



 Variable containing:
 1.1982
[torch.FloatTensor of size 1]
 

bbbeth you thine,
And your bear the restrembersh, the kix's, better life:
And me with the wirst. Wemp julition.

CAMILLO:
Your worth gener Richard! be his swert to his
so false he rights in bids your s



 Variable containing:
 1.7775
[torch.FloatTensor of size 1]
 

both being encessures not
Signiobs friend thy suffer, who molour so my some will,
And her wons, when, prospets their say
As desert! O, it down for rang to this never:
You thus any and years in his frie



 Variable containing:
 1.5648
[torch.FloatTensor of size 1]
 

burry elder and
To pearing to in them your d